<a href="https://colab.research.google.com/github/kevinscaria/InstructABSA/blob/main/ATE_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece
  !pip install transformers[torch]

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/InstructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

<ipython-input-3-925b459844e6>:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  use_mps = True if torch.has_mps else False


In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

In [ ]:
!pip install peft

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
from peft import LoraConfig
from peft import LoraModel

In [ ]:
class T5GeneratorWithLoRA(T5Generator):
    def __init__(self, model_checkpoint, lora_config):
        # Initialize the base T5Generator
        super().__init__(model_checkpoint)

        # Apply LoRA to the T5 model
        self.model = LoraModel(self.model, **lora_config)

## Training

In [23]:
task_name = 'ate'
experiment_name = 'lapt2014_iabsa1_flant5_instruct_base'
# model_checkpoint = 'allenai/tk-instruct-base-def-pos'
model_checkpoint = 'pszemraj/flan-t5-base-instructiongen'  # Replace with the correct model checkpoint

print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1_flant5_instruct_base
Model output path:  ./Models/ate/pszemrajflan-t5-base-instructiongen-lapt2014_iabsa1_flant5_instruct_base


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)


# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()


train_df, eval_df = train_test_split(id_tr_df, test_size=0.2, random_state=42)  # for example, 20% for evaluation

loader = DatasetLoader(train_df, id_te_df, eval_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.val_df_id is not None:
    loader.val_df_id = loader.create_data_in_ate_format(loader.val_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

In [26]:
t5_exp = T5Generator(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
target_modules = ["q", "v"]
lora_config = LoraConfig(
r=16,
target_modules = target_modules,
lora_alpha=8,
lora_dropout=0.05,
bias="none",
task_type="CAUSAL_LM",)

In [ ]:
from peft import get_peft_model

In [ ]:
print(t5_exp.model)

In [ ]:
t5_exp.model = get_peft_model(t5_exp.model, lora_config)
t5_exp.model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096414524241463


In [27]:
# Create T5 utils object

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Map:   0%|          | 0/2436 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/609 [00:00<?, ? examples/s]

In [ ]:
print(t5_exp.model)

In [28]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.252412
2,0.666300,0.199023
3,0.666300,0.220867
4,0.228200,0.222029


In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

## Inference

In [29]:
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

In [30]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test',  batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 191/191 [01:16<00:00,  2.49it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:20<00:00,  2.47it/s]


#Original

In [13]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.9179238046535413
Train Recall:  0.9169859514687101
Train F1:  0.917454638384871
Test Precision:  0.9320197044334976
Test Recall:  0.9166666666666666
Test F1:  0.9242794333170493


#Flan T5 small Instruct

In [22]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.8835341365461847
Train Recall:  0.842911877394636
Train F1:  0.8627450980392158
Test Precision:  0.8960905349794238
Test Recall:  0.8439922480620154
Test F1:  0.8692614770459081


#Flan T5 base instruct

In [31]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.9366829018200461
Train Recall:  0.9333333333333333
Train F1:  0.935005117707267
Test Precision:  0.9302554027504911
Test Recall:  0.9176356589147286
Test F1:  0.9239024390243902


In [ ]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.9221967963386728
Train Recall:  0.9264367816091954
Train F1:  0.9243119266055047
Test Precision:  0.9243452958292919
Test Recall:  0.9234496124031008
Test F1:  0.9238972370334463


In [ ]:
p, r, f1,_ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1,_ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7013412816691506
Train Recall:  0.6010217113665389
Train F1:  0.6473177441540577
Test Precision:  0.6756756756756757
Test Recall:  0.5813953488372093
Test F1:  0.6250000000000001
